In [18]:
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras 

import pickle


import warnings
warnings.filterwarnings('ignore')

# Drawing helpers
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose


In [11]:
# Prepare for processing input
IMPORTANT_LMS = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE"
]

headers = ["label"]  # Label column

for lm in IMPORTANT_LMS:
    headers += [f"{lm.lower()}_x", f"{lm.lower()}_y",
                f"{lm.lower()}_z", f"{lm.lower()}_v"]

In [12]:
from math import sqrt


def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent / 100)
    height = int(frame.shape[0] * percent / 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation=cv2.INTER_AREA)


def calculate_distance(pointX, pointY) -> float:
    '''
    Calculate a distance between 2 points
    '''

    x1, y1 = pointX
    x2, y2 = pointY

    return sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def extract_important_keypoints(results) -> list:
    '''
    Extract important keypoints from mediapipe pose detection
    '''
    landmarks = results.pose_landmarks.landmark

    data = []
    for lm in IMPORTANT_LMS:
        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
        data.append([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])

    return np.array(data).flatten().tolist()



In [13]:
df = pd.DataFrame(
    columns=["video", "stage", "shoulder_width", "feet_width", "knee_width"])

df


,video,stage,shoulder_width,feet_width,knee_width


In [14]:
VIDEO_PATH = "./ds_video/squat_demo.mp4"

In [23]:

cap = cv2.VideoCapture(VIDEO_PATH)

current_stage = ""
prediction_probability_threshold = 0.8

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, image = cap.read()

        if not ret:
            break

        # Reduce size of a frame
        image = rescale_frame(image, 50)
        # image = cv2.flip(image, 1)

        video_dimensions = [image.shape[1], image.shape[0]]

        # Recolor image from BGR to RGB for mediapipe
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        # Calculate and compare the distance of 2 ankle and shoulder width
        if not results.pose_landmarks:
            print("No human found")
            continue

        # Recolor image from BGR to RGB for mediapipe
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw landmarks and connections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(
            244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        shoulder_width = feet_width = None

        try:
            # Extract keypoints from frame for the input
            row = extract_important_keypoints(results)
            X = pd.DataFrame([row], columns=headers[1:])
            X = pd.DataFrame(input_scaler.transform(X))

            # Make prediction and its probability
            predicted_class = trained_model.predict(X)[0]
            prediction_probability = trained_model.predict_proba(X)[0]

            # Evaluate model prediction for the current stage
            if prediction_probability[prediction_probability.argmax()] >= prediction_probability_threshold:
                if predicted_class == "down":
                    current_stage = "down"
                elif current_stage == "middle" and predicted_class == "up":
                    current_stage = "up"
            else:
                current_stage = "middle"

            # Calculate and compare the distance of 2 ankle and shoulder width
            landmarks = results.pose_landmarks.landmark

            # * Calculate shoulder width
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

            shoulder_width = calculate_distance(left_shoulder, right_shoulder)

            # * Calculate 2-foot width
            left_foot_index = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,
                               landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
            right_foot_index = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,
                                landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]

            feet_width = calculate_distance(left_foot_index, right_foot_index)

            # * Calculate 2-knee width
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]

            knee_width = calculate_distance(left_knee, right_knee)
            

            # Visualization
            # Status box
            cv2.rectangle(image, (0, 0), (500, 60), (245, 117, 16), -1)

            # Display feet distance
            cv2.putText(image, "FEET", (15, 12),
                        cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(feet_width, 2)), (10, 40),
                        cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Display shoulder distance
            cv2.putText(image, "SHOULDER", (95, 12),
                        cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(shoulder_width, 2)), (90, 40),
                        cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Display knee distance
            cv2.putText(image, "KNEE", (205, 12),
                        cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(knee_width, 2)), (200, 40),
                        cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Display knee distance
            cv2.putText(image, "STAGE", (305, 12),
                        cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, current_stage, (300, 40),
                        cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        except Exception as e:
            print(f"Error: {e}")

        cv2.imshow("CV2", image)

        # Pressed key for action
        k = cv2.waitKey(1) & 0xFF

        # Press C to add to df
        if k == ord('c'):
            df = df.append({"video": VIDEO_PATH, "stage": current_stage, "shoulder_width": shoulder_width,
                           "feet_width": feet_width, "knee_width": knee_width}, ignore_index=True)

        # Press q to stop
        elif k == ord("q"):
            break
        else:
            continue

    cap.release()
    cv2.destroyAllWindows()

    # (Optional)Fix bugs cannot close windows in MacOS (https://stackoverflow.com/questions/6116564/destroywindow-does-not-close-window-on-mac-using-python-and-opencv)
    for i in range(1, 5):
        cv2.waitKey(1)


Error: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- left_ankle_v
- left_ankle_x
- left_ankle_y
- left_ankle_z
- left_knee_v
- ...
Feature names seen at fit time, yet now missing:
- left_elbow_v
- left_elbow_x
- left_elbow_y
- left_elbow_z
- left_wrist_v
- ...

Error: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- left_ankle_v
- left_ankle_x
- left_ankle_y
- left_ankle_z
- left_knee_v
- ...
Feature names seen at fit time, yet now missing:
- left_elbow_v
- left_elbow_x
- left_elbow_y
- left_elbow_z
- left_wrist_v
- ...

Error: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- left_ankle_v
- left_ankle_x
- left_ankle_y
- left_ankle_z
- left_knee_v
- ...
Feature names seen at fit time, yet now missing:
- left_elbow_v
- left_elbow_x
- left_elbow_y
- left_elbow_z
- left_wrist_v
- ...

Error: The feature names should match those th

NameError: name 'knee_width' is not defined

: 

In [ ]:
df

,video,stage,shoulder_width,feet_width,knee_width
